In [ ]:
#importing libraries
import joblib
import librosa
import numpy as np

In [ ]:
#loading saved model
bundle = joblib.load("audio_classifier.pkl")
model = bundle["model"]
scaler = bundle["scaler"]
svd = bundle["svd"]

In [ ]:
#function to extract features
def extract_features(file_path):
    y, sr = librosa.load(file_path, duration=5.0, sr=None)
    zcr = librosa.feature.zero_crossing_rate(y).mean()
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = mfcc.mean(axis=1)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr).mean(axis=1)
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr).mean(axis=1)
    features = np.hstack([zcr, centroid, bandwidth, tempo, mfccs_mean, chroma[:2], tonnetz[:2]])
    return features


In [ ]:
#function to make predictions
def predict_audio(file_path):
    features = extract_features(file_path).reshape(1, -1)
    X_scaled = scaler.transform(features)
    X_svd = svd.transform(X_scaled)
    pred = model.predict(X_svd)[0]
    return pred

#predicting on new samples
samples = [
    "sample_dog.wav",
    "sample_drill.wav",
    "sample_street.wav"
]

#printing predictions 
for s in samples:
    print(f"{s} → Predicted class: {predict_audio(s)}")
